In [ ]:
import numpy as np
import pandas as pd
import importlib
import matplotlib.pyplot as plt
from pathlib import Path
import os
import string
import random
from sklearn.gaussian_process.kernels import RBF, Matern, ConstantKernel
from scipy import special
import sys

sys.path.insert(0, '..')

In [ ]:
from modules import data_generator as dg
from modules import regression as rg

importlib.reload(dg)


t_max = 2500
delay = 75
sampling_rate = 0.08
relative_error = 0.025
n_gaps = 5
gap_size = 10
image_ratio = 1
seed = 0

In [ ]:
plt.rcParams['figure.figsize'] = (10.0, 8.0)
plt.rcParams['text.usetex'] = True

datagen = dg.DataGenerator(t_max=t_max, 
                           delay=delay, 
                           sampling_rate=sampling_rate, 
                           image_ratio=image_ratio, 
                           n_gaps=n_gaps, 
                           gap_size=gap_size,
                           relative_err=relative_error,
                           seed=seed)

In [ ]:
means, sigmas, heights = datagen.generate_gaussian_signal_parameters()
#heights = np.ones(means.shape)
t_domain = datagen.generate_noisy_time_domain()
t_supersampled = datagen.get_supersampled_timedomain()

dataset = datagen.generate_single_realization_dataset(means=means, 
                                                      sigmas=sigmas, 
                                                      heights=heights, 
                                                      gap_size=gap_size)

#offset = (dataset["A"].max()-dataset["A"].min())/2
offset = 0
fig, ax = plt.subplots()

ax.errorbar(dataset["time"], dataset["A"]+offset, yerr=dataset["sigmaA"], label="A",
            color="blue", linestyle="None", marker='o', capsize=5, markersize=5)
ax.errorbar(dataset["time"], dataset["B"], yerr=dataset["sigmaB"], label="B",
            color="red", linestyle="None", marker='o', capsize=5, markersize=5)
ax.plot(t_supersampled, datagen.get_underlying_signal(t_supersampled, image="A") + offset, 
        color="blue", linestyle="dashed")
ax.plot(t_supersampled, datagen.get_underlying_signal(t_supersampled, image="B"), 
        color="red", linestyle="dashed")
ax.legend(fontsize=15)

In [ ]:
t = dataset['time'].values
yA = dataset['A'].values
yB = dataset['B'].values
errA = dataset['sigmaA']
errB = dataset['sigmaB']

yA = (yA - yA.mean()) / yA.std()
yB = (yB - yB.mean()) / yB.std()


kernel = ConstantKernel(2, (1e-3, 1e2)) * Matern(length_scale=200.0, length_scale_bounds=(1, 300), nu=1.5)

gp1 = rg.fit_GP_to_lightcurve(t, yA, errA, kernel)
gp2 = rg.fit_GP_to_lightcurve(t, yB, errB, kernel)

In [ ]:
gp_step = 0.2
support = np.arange(t[0] - 5e1, t[-1] + 5e1, gp_step)
y_pred1, sigma1 = gp1.predict(np.expand_dims(support,1), return_std=True)
y_pred2, sigma2 = gp2.predict(np.expand_dims(support,1), return_std=True)

In [ ]:

plt.figure()
plt.plot(t, yB, 'g.', markersize=3, label='MontecarloA')
plt.plot(t, yA, 'b.', markersize=3, label='MontecarloB')
plt.plot(support, y_pred1, 'b-', label='PredictionA')
plt.plot(support, y_pred2, 'g-', label='PredictionB')
plt.fill_between(support, y_pred1 - special.erfinv(0.95)*sigma1, y_pred1 + special.erfinv(0.95)*sigma1,
         alpha=.5, facecolor='b', label='95% confidence interval B')
plt.fill_between(support, y_pred2 - special.erfinv(0.95)*sigma2, y_pred2 + special.erfinv(0.95)*sigma2,
         alpha=.5, facecolor='g', label='95% confidence interval A')
plt.xlabel('$t$')
plt.ylabel('$f(t)$')
plt.legend()

In [ ]:
delay = rg.time_delay_grid_search(y_pred1, y_pred2, sigma1, sigma2, gp_step,
                                  dt_min=0, dt_max=200)
print('Estimated time delay: %s days' % delay)

In [ ]:
def numeric_derivative(f: np.ndarray, step: float) -> np.ndarray:
    return (f[1:] - f[:-1]) / step


def WAV(f, sigma, step):
    f_prime = numeric_derivative(f, step)
    weights = 2 / (sigma[:-1] + sigma[1:])
    WAV = np.dot(np.abs(f_prime), weights) / weights.sum()
    return WAV


In [ ]:
dt_min = 0
dt_max = 200

shift = np.arange(int(dt_min/gp_step), int(dt_max/gp_step), 1)
win = len(shift)
WAV_values = []
for i in shift:
    diff = y_pred1[win:-win] - y_pred2[win + i:-win + i]
    sigma_diff = sigma1[win:-win] + sigma2[win + i:-win + i]
    WAV_values.append(WAV(diff, sigma_diff, gp_step))

In [ ]:
plt.plot(WAV_values)